In [ ]:
from data import *
from transformer import *

twitterData, twitterTokenizers = loadTwitterThreadData('twitterThreads/')
inputVocabSize = len(twitterTokenizers[0].wordMap)
targetVocabSize = len(twitterTokenizers[1].wordMap)
print()
print("Vocabulary size:", inputVocabSize, ", ", targetVocabSize)

In [ ]:
conversationData, conversationTokenizers = loadConversationData()
inputVocabSize = len(conversationTokenizers[0].wordMap)
targetVocabSize = len(conversationTokenizers[1].wordMap)

In [ ]:
# data, tokenizers = loadStableData(100)
#
# inputVocabSize = tokenizers.pt.get_vocab_size().numpy()
# targetVocabSize = tokenizers.en.get_vocab_size().numpy()

In [ ]:
numLayers = 4
embeddingDepth = 128
feedDepth = 512
numHeads = 8

optimizer = tf.keras.optimizers.Adam(CustomSchedule(embeddingDepth), beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

model = Transformer(numLayers=numLayers, inputVocabSize=inputVocabSize, targetVocabSize=targetVocabSize, embeddingDepth=embeddingDepth,
                    feedDepth=feedDepth, numHeads=numHeads)
model.compile(loss=masked_loss, optimizer=optimizer, metrics=[masked_accuracy])

In [ ]:
# model.fit(conversationData, epochs=40)
model.fit(twitterData, epochs=20)

In [ ]:
# from datetime import datetime
# now = datetime.now()
# timeText = now.strftime("%y.%m.%d.%H.%M.%S")
# model.export("model " + timeText)

In [ ]:
from transformer import Chat
chat = Chat(twitterTokenizers, model)

In [ ]:
while True:
    prompt = input("Chat with Twitter: ")
    print(prompt)
    output = chat(prompt)
    print(output)